In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import pretrainedmodels as ptmodels
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy_b, log_loss
from pipeline.model_methods import validate_img, train_img, validate_vid, train_vid
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset, strong_aug
from pipeline.image_extracting import extract_faces, FastMTCNN

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid, log_loss_b, accuracy_b_mean
from pipeline.model_methods import validate_vid_bf, train_vid_bf
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace
from network.lstm_deepfake import LSTMDF

from efficientnet_pytorch import EfficientNet

from pytorchcv.model_provider import get_model
from catalyst.dl import SupervisedRunner
from catalyst.contrib.nn import OneCycleLRWithWarmup


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)
alchemy not available, to install alchemy, run `pip install alchemy-catalyst`.


In [2]:
y_train = pd.read_csv(r'D:\Machine Learning\deepfake-detection\data\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(100000, 19154)

In [3]:
#%%script false

batch_size = 16
normalize = torchvision.transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

x_train = load_img_dataset(r'data\train2', batch_size, resize=224, normalize=normalize)
x_val = load_img_val_dataset('data\img_val', 10, resize=224, normalize=normalize)

x_train_vid = r'D:\Machine Learning\deepfake-detection\data\train_set'
x_val_vid = r'data\val'
x_test_vid = r'D:\Machine Learning\deepfake-detection\data\cross_val'

In [4]:
#model = EfficientNet.from_name('efficientnet-b0')
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=1)
#model = models.resnext50_32x4d(pretrained=True) 
#model =  models.resnext101_32x8d(pretrained=True) 
#model =  MyResNeXt()
#model = models.resnet18(pretrained=True)
#model._fc = nn.Linear(model._fc.in_features, 1)
#model.fc = nn.Linear(model.fc.in_features, 1)

Loaded pretrained weights for efficientnet-b2


In [5]:
%%script false
found = False
for child in model.named_parameters():  
    #'''
    if child[0] == 'layer4.0.conv2.weight':
        found = True
    #'''
    child[1].requires_grad = found

Couldn't find program: 'false'


In [6]:
model = model.to(device)
loss = torch.nn.BCELoss()

#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.) 
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.) 
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 0.001, 0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(x_train), epochs=10)

#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: epoch // 30, lambda epoch: 0.95 ** epoch], last_epoch=-1)

In [7]:
[i[0] for i in model.named_parameters() if i[1].requires_grad]

['_conv_stem.weight',
 '_bn0.weight',
 '_bn0.bias',
 '_blocks.0._depthwise_conv.weight',
 '_blocks.0._bn1.weight',
 '_blocks.0._bn1.bias',
 '_blocks.0._se_reduce.weight',
 '_blocks.0._se_reduce.bias',
 '_blocks.0._se_expand.weight',
 '_blocks.0._se_expand.bias',
 '_blocks.0._project_conv.weight',
 '_blocks.0._bn2.weight',
 '_blocks.0._bn2.bias',
 '_blocks.1._depthwise_conv.weight',
 '_blocks.1._bn1.weight',
 '_blocks.1._bn1.bias',
 '_blocks.1._se_reduce.weight',
 '_blocks.1._se_reduce.bias',
 '_blocks.1._se_expand.weight',
 '_blocks.1._se_expand.bias',
 '_blocks.1._project_conv.weight',
 '_blocks.1._bn2.weight',
 '_blocks.1._bn2.bias',
 '_blocks.2._expand_conv.weight',
 '_blocks.2._bn0.weight',
 '_blocks.2._bn0.bias',
 '_blocks.2._depthwise_conv.weight',
 '_blocks.2._bn1.weight',
 '_blocks.2._bn1.bias',
 '_blocks.2._se_reduce.weight',
 '_blocks.2._se_reduce.bias',
 '_blocks.2._se_expand.weight',
 '_blocks.2._se_expand.bias',
 '_blocks.2._project_conv.weight',
 '_blocks.2._bn2.weight',


In [8]:
0.37011126635215613 * 100 / 83, 0.37011126635215613 + 0.08228

(0.4459171883760917, 0.45239126635215615)

In [9]:
#%%script false
# 38% / 47% / 57% /58.5% / 61% - 0.656% (0.99344%) - video
# 83% undersampling / 50% oversampling / 89.8% original dataset / 0.08228

metric, lost = train_img(model, loss, optimizer, scheduler, x_train, x_val, y_train, accuracy_b, device, [log_loss_sigmoid, accuracy_sigmoid_mean], None, 
                          epochs=10, 
                       batch_size=batch_size, 
                       del_net=False, 
                       useInference=False, 
                       inference=False, 
                       useScheduler=False,
                       checkpoint=[0.37, 0.99],
                         reverse=False
                      )


Validation: metrics  0.7435030113026978 loss  0.5686379752635513
Train: metrics  0.7296028880866426 loss  0.5278959004295862
Expected LB value: 0.6509179752635513 0.6851059942934353
Epoch: 1



Validation: metrics  0.788218793828892 loss  0.4587039594421039
Train: metrics  0.8235108303249098 loss  0.3851020627369304
Expected LB value: 0.5409839594421039 0.552655372821812
Epoch: 2



Validation: metrics  0.636911145945054 loss  1.0393574458377257
Train: metrics  0.8593185920577617 loss  0.3182312691537159
Expected LB value: 1.1216374458377256 1.2522378865514767
Epoch: 3



Validation: metrics  0.7947364078871381 loss  0.48183109777313404
Train: metrics  0.8795803249097472 loss  0.2786013092159795
Expected LB value: 0.5641110977731341 0.5805193949073905
Epoch: 4


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "d:\users\user\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-a375b780ae36>", line 13, in <module>
    reverse=False
  File "C:\Users\user\Desktop\Projects\dfdc-kaggle-solution\pipeline\model_methods.py", line 183, in train_img
    loss_value.backward()
  File "d:\users\user\appdata\local\programs\python\python36\lib\site-packages\torch\tensor.py", line 195, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "d:\users\user\appdata\local\programs\python\python36\lib\site-packages\torch\autograd\__init__.py", line 99, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\users\user\appdata\local\programs\python\python36

KeyboardInterrupt: 

In [ ]:
from operator import itemgetter
min_lost = max(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = min(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
%%script false

# default - Validation: metrics  0.6033333333333334 loss:  0.6769222617149353

validate_vid_bf(model, x_val_vid, y_train, log_loss_sigmoid, accuracy_sigmoid_mean, device, 2, face_extractor, 
                fast_mtcnn=fast_mtcnn,
                print_results=True, 
                reverse=False)